In [ ]:
Este es el ejemplo D’Aertrycke et al. (2017), con el solver de gurobi

In [ ]:
import Pkg
Pkg.add("Gurobi")

In [ ]:
using JuMP,Gurobi
A=[300,350,400,450,500];
theta=[0.2,0.2,0.2,0.2,0.2];
B=1;
I=90; 
C=60;
tau=8760;


model=Model(Gurobi.Optimizer)

#Variables
@variable(model, x>=0)  #Capacidad
@variable(model, Y[i in 1:5]>=0) # Produccion en escenario w
@variable(model,Q[i in 1:5]>=0) # Demanda en escenario w

#Restricciones
@constraint(model, demanda[i in 1:5], Y[i] == Q[i] )
@constraint(model, capacidad[i in 1:5], Y[i]-x<=0.0)

#FO
@objective(model, Min, I*1000*x + tau*(sum(theta[i]*(C*Y[i] - A[i]*Q[i] + B*0.5*(Q[i]^2)) for i in 1:5) ))



In [ ]:
#Resolver
JuMP.optimize!(model)
     


println("Q = ", JuMP.value.(Q))
Qlist=JuMP.value.(Q)
     
j=1
P=[]
while j<6
    Pi=A[j]-B*Qlist[j]
    append!(P, Pi) 
    j=j+1
end
println("P = ",P)
println("x = ", JuMP.value(x))

In [ ]:
Ahora vamos a ver lo que es el computo del mismo ejemplo, pero con el solver de Ipopt.jl.

In [ ]:
import Pkg
Pkg.add("Ipopt")

In [ ]:
using JuMP, Ipopt
A = [300, 350, 400, 450, 500]
theta = [0.2, 0.2, 0.2, 0.2, 0.2]
B = 1
I = 90
C = 60
tau = 8760

model = Model(Ipopt.Optimizer)

# Variables
@variable(model, x >= 0)  # Capacidad
@variable(model, Y[1:5] >= 0) # Producción en escenario w
@variable(model, Q[1:5] >= 0) # Demanda en escenario w

# Restricciones
@constraint(model, demanda[i = 1:5], Y[i] == Q[i])
@constraint(model, capacidad[i = 1:5], Y[i] - x <= 0)

# Función Objetivo
@objective(model, Min, I*1000*x + tau * sum(theta[i] * (C * Y[i] - A[i] * Q[i] + B * 0.5 * Q[i]^2) for i = 1:5))

In [ ]:
optimize!(model)

# Resultados
println("Q = ", value.(Q))
Qlist = value.(Q)

j = 1
P = []
while j < 6
    Pi = A[j] - B * Qlist[j]
    append!(P, Pi)
    j = j + 1
end
println("P = ", P)
println("x = ", value(x))

In [ ]:
En lo que estoy teniendo problemas es en poder hacerlo con el solver Path... 
No se si esta bien pasado a un problema de MCP y tambien no se si este modelo con la f.o. cuadratica se puede resolver como MCP


In [ ]:
using Pkg
Pkg.add("PATHSolver")

In [ ]:
using JuMP, PATHSolver

# Configurar licencia para PATH
PATHSolver.c_api_License_SetString("2830898829&Courtesy&&&USR&45321&5_1_2021&1000&PATH&GEN&31_12_2025&0_0_0&6000&0_0")

function modeloRNcompetitivecapacityequilibrium(cant_esce, a, theta, B, tau, c, I)
    model = Model(PATHSolver.Optimizer)
    
    # Decision variables x, Y, Q
    @variable(model, x >= 0)  # Capacidad
    @variable(model, Y[1:cant_esce], Int)
    @variable(model, Q[1:cant_esce], Int)
    
    # Restricciones
    for i in 1:cant_esce
        @constraint(model, Y[i] == Q[i])
        @constraint(model, Y[i] <= x)
    end
    
    # F.O
    @NLobjective(model, Min, I * 1000 * x + tau * sum(theta[i] * (c * Y[i] - a[i] * Q[i] + (B / 2) * (Q[i] * Q[i])) for i in 1:cant_esce))
    
    optimize!(model)
    
    P = []
    IM = []
    for i in 1:cant_esce
        push!(P, a[i] - B * value(Q[i]))
        push!(IM, (tau * (P[i] - c) * value(Y[i]) - I * value(x)) / value(x))
    end
    
    Qprom = sum(value(Q[i]) for i in 1:cant_esce)
    
    println("Q Promedio = ", Qprom / cant_esce)
    println("x = ", value(x))
    println("Valor función objetivo = ", objective_value(model))
    
    println(cant_esce, a, theta, B, tau, c, I)
    
    println("\nQ, Y, P and Investment Margin values:")
    for i in 1:cant_esce
        println("Q($i) = ", value(Q[i]), "    Y($i) = ", value(Y[i]), "    P($i) = ", P[i], "    Investment Margin = ", IM[i])
    end
end

cant = 5
A = [300, 350, 400, 450, 500]
Theta = [0.2, 0.2, 0.2, 0.2, 0.2]
B = 1  # $/MWh
tau = 8760  # duracion por segmento por hora
C = 60  # costo de operacion en $/mWhora
I = 90  # capital para gasto

modeloRNcompetitivecapacityequilibrium(cant, A, Theta, B, tau, C, I)

In [ ]:
Pruebo con complementarity.jl pero tampoco me funciona 

In [ ]:
using Pkg
Pkg.add("Complementarity")

In [ ]:
using JuMP, PATHSolver, Complementarity

function modeloRNcompetitivecapacityequilibrium(cant_esce, a, theta, B, tau, c, I)
    model = MCPModel()
    
    # Decision variables x, Y, Q
    @variable(model, x >= 0)  # Capacidad
    @variable(model, Y[1:cant_esce] >= 0)
    @variable(model, Q[1:cant_esce])
    
    # Restricciones y condiciones de complementariedad
    for i in 1:cant_esce
        @constraint(model, Q[i] == Y[i])
        @complements(0 <= Y[i], Y[i] <= x)
    end
    
    
    resolver = solveMCP(model)
    
    P = []
    IM = []
    for i in 1:cant_esce
        push!(P, a[i] - B * value(Q[i]))
        push!(IM, (tau * (P[i] - c) * value(Y[i]) - I * value(x)) / value(x))
    end
    
    Qprom = sum(value(Q[i]) for i in 1:cant_esce)
    
    println("Q Promedio = ", Qprom / cant_esce)
    println("x = ", value(x))
    
    println(cant_esce, a, theta, B, tau, c, I)
    
    println("\nQ, Y, P and Investment Margin values:")
    for i in 1:cant_esce
        println("Q($i) = ", value(Q[i]), "    Y($i) = ", value(Y[i]), "    P($i) = ", P[i], "    Investment Margin = ", IM[i])
    end
end

cant = 5
A = [300, 350, 400, 450, 500]
Theta = [0.2, 0.2, 0.2, 0.2, 0.2]
B = 1  # $/MWh
tau = 8760  # duracion por segmento por hora
C = 60  # costo de operacion en $/mWhora
I = 90  # capital para gasto

modeloRNcompetitivecapacityequilibrium(cant, A, Theta, B, tau, C, I)
